# 2) Assign Map Colors

For a new map after redistricing, try to make the colors match when switching between map versions.

Goal: If 1C01 is light blue on the 2012 map, then the same area should also be light blue on the 2022 map. The district number might have changed, though! 

In [1]:
import pandas as pd
import geopandas as gp

In [2]:
districts = pd.read_csv('../data/districts.csv')
overlap = pd.read_csv('overlap_forwards.csv')

In [3]:
# Should be 345
map_2022 = gp.read_file('to-mapbox-2022-smd-data-overlap.geojson')
len(map_2022)

345

In [4]:
# Should be 296 - the new district that each old district most overlaps
overlap_top_forwards = overlap[overlap.district_rank == 1].copy()
len(overlap_top_forwards)

296

There are situations where two old districts overlap the most with the same new district. For instance, the old 1C02 and 1C04 both overlap the most with the new 1C04. In these cases, we want to "connect" the old district to the new district where both share the most overlap area. 

In [5]:
overlap_top_forwards['overlap_area_rank'] = (
    overlap_top_forwards.groupby('smd_id_2022').overlap_area.rank(ascending=False)
)
overlap_top_forwards[overlap_top_forwards['smd_id_2022'] == 'smd_2022_1C04']

,smd_id_2022,district_area_2022,smd_id_2012,district_area_2012,overlap_area,overlap_perc,district_rank,overlap_area_rank
195,smd_2022_1C04,330058.746971,smd_1C02,179011.812029,110259.640709,0.615935,1.0,2.0
206,smd_2022_1C04,330058.746971,smd_1C04,274709.409745,218364.638044,0.794893,1.0,1.0


In [6]:
overlap_top_match = overlap_top_forwards[overlap_top_forwards.overlap_area_rank == 1].copy()

In [7]:
# Left join the 2022 map to the 2012 district overlap
district_overlap = pd.merge(
    districts[['smd_id', 'map_color_id']], overlap_top_match, how='left', left_on='smd_id', right_on='smd_id_2012'
)

# Drop this duplicate column to avoid confusion and collision
district_overlap.drop(columns=['smd_id'], inplace=True)

# Should be 296
len(district_overlap)

296

In [8]:
map_2022_colors = map_2022.merge(
    district_overlap, how='left', left_on='smd_id', right_on='smd_id_2022'
)

# Should be 345
len(map_2022_colors)

345

In [9]:
map_2022_colors['map_color_id'] = map_2022_colors['map_color_id'].fillna(13)

In [10]:
# Make sure this column in an integer. Because it was previously NULL, pandas set it a decimal
map_2022_colors['map_color_id'] = map_2022_colors['map_color_id'].astype('int64')

In [11]:
map_2022_colors['qgis_label'] = '2022_' + map_2022_colors['smd_name']

In [12]:
map_2022_colors.head()

,smd_id,smd_name,anc_id,geometry,map_color_id,smd_id_2022,district_area_2022,smd_id_2012,district_area_2012,overlap_area,overlap_perc,district_rank,overlap_area_rank,qgis_label
0,smd_2022_1C08,1C08,1C,"POLYGON ((-77.03649 38.92274, -77.03663 38.922...",8,smd_2022_1C08,128342.785633,smd_1C08,139664.389254,92014.851455,0.658828,1.0,1.0,2022_1C08
1,smd_2022_1C07,1C07,1C,"POLYGON ((-77.03918 38.92494, -77.03921 38.924...",7,smd_2022_1C07,129160.040163,smd_1C07,203187.796445,111510.190211,0.548804,1.0,1.0,2022_1C07
2,smd_2022_1C03,1C03,1C,"POLYGON ((-77.04269 38.92267, -77.04282 38.922...",3,smd_2022_1C03,119796.170816,smd_1C03,119796.342840,119796.031763,0.999997,1.0,1.0,2022_1C03
3,smd_2022_1C02,1C02,1C,"POLYGON ((-77.04478 38.91906, -77.04484 38.919...",13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022_1C02
4,smd_2022_1C01,1C01,1C,"POLYGON ((-77.04168 38.91875, -77.04180 38.918...",1,smd_2022_1C01,135254.012713,smd_1C01,154959.758434,127584.174939,0.823337,1.0,1.0,2022_1C01


In [13]:
map_2022_colors.groupby('map_color_id').size()

map_color_id
1     23
2     23
3     25
4     23
5     21
6     21
7     20
8     23
9     22
10    22
11    19
12    21
13    82
dtype: int64

In [14]:
# Should be 345
len(map_2022_colors)

345

In [15]:
map_columns = ['smd_id', 'smd_name', 'anc_id', 'geometry']

map_2022_colors[map_columns + ['map_color_id', 'qgis_label']].to_file(
    'to-mapbox-2022-smd-data-colors.geojson', driver='GeoJSON')

In [16]:
# no_geo = [c for c in map_2022_overlap.columns if c != 'geometry']
# map_2022_overlap[no_geo].sort_values(by='smd_id').to_clipboard()